# Titanicデータセットを用いた機械学習入門

<div style="display:flex;justify-content:center;">
    <img src="./assets/RMS_Titanic_3.jpg"  width="600px" alt="https://commons.wikimedia.org/wiki/File:RMS_Titanic_3.jpg" />
</div>

今回は、データサイエンスや機械学習を学ぶものの多くが通る道である、Titanicデータセットを用いたデータ分析を行っていきます。

もちろんタイタニックは、映画「タイタニック」で有名なあの船。

> タイタニック（英語: RMS Titanic、ロイヤルメールシップ・タイタニック）は、20世紀初頭に建造されたイギリス船籍のオーシャン・ライナー。
> ホワイト・スター・ライン社が保有するオリンピック級客船の2番船であったが、処女航海中の1912年4月14日深夜に氷山に衝突し、その際の損傷による浸水が原因となって翌15日未明に沈没した。([wikipediaより](https://ja.wikipedia.org/wiki/%E3%82%BF%E3%82%A4%E3%82%BF%E3%83%8B%E3%83%83%E3%82%AF_(%E5%AE%A2%E8%88%B9)))

## Titanic dataset について

Titanic datasetは色々なところでフリーのデータセットとして公開されています。
- Kaggle (https://www.kaggle.com/competitions/titanic)
- Tensorflow Datasets (https://www.tensorflow.org/datasets/catalog/titanic?hl=en)
- Seaborn Datasets (https://github.com/mwaskom/seaborn-data)
などなど

今回は、 Kaggleのものを利用します。

## 機械学習プロジェクトの一般的な流れ

1. 分析の目的と問題設定
2. データを取得する
3. EDA ~ データからインサイトを得る ~
4. 前処理 ~ データクレンジングとフィーチャーエンジニアリング ~
5. モデルの作成、学習、推論の実行
<br>↑今回はここまで
6. モデルのファインチューニング
7. 結果の提示
8. システムに組み込む

# 必要なライブラリのインポート

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

## データを取得する

早速データを取得してみます。

In [3]:
test_set = pd.read_csv('./datasets/test.csv')
train_set = pd.read_csv('./datasets/train.csv')

In [4]:
# データを頭から数行だけ見てみる
train_set.head() # head(10)とすると10行取得できる

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
print('train:', train_set.columns)
print('test:', test_set.columns)

train: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
test: Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


test_setには`Survived`が存在しない。

各列の説明はこの通り

| 列名 | 説明 |
|------|------|
| PassengerId | 乗客のID |
| Survived | 生存したか否か (0 = No, 1 = Yes) |
| Pclass | チケットのクラス (1 = 1st, 2 = 2nd, 3 = 3rd) |
| Name | 乗客の名前 |
| Sex | 性別 |
| Age | 年齢 |
| SibSp | タイタニック号に乗船していた兄弟/配偶者の数 |
| Parch | タイタニック号に乗船していた親/子供の数 |
| Ticket | チケット番号 |
| Fare | 旅客運賃 |
| Cabin | キャビン番号 |
| Embarked | 乗船港 (C = Cherbourg, Q = Queenstown, S = Southampton) |


train_setのデータを用いて、各列の情報からSurvivedを予測するモデルを作成する。

test_setのデータのそれぞれの行のSurvivedの結果を予測する。

という流れになる。

## 注意点

Kaggle の test.csvの真の答えは、Kaggleが持っていて、Kaggle上で予測結果を提出することでしか、最終的な予測の精度は測れない（Web上に答えは転がっているだろうが)。

ここでは、train.csvの一部のデータ(例えば全体の20%)を答えがわからないふりをして傍に置いておき、

もう一方のデータを使って学習、とっておいたデータ(から真の`Survived`を排除したデータ)で予測を行い、精度を測ることにする。

以下はそのために、データを分けている。

In [9]:
train_data, test_data = train_test_split(train_set, test_size=0.2)

test_survived = test_data[["Survived"]]
test_data = test_data.drop('Survived', axis=1)

print('train_data:', train_data.columns)
print()
print('test_data:', test_data.columns)
print()
print('test_survived:', test_survived.columns)

train_data: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

test_data: Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

test_survived: Index(['Survived'], dtype='object')


これで、

train_data ・・・ 学習用のデータ<br>
test_data ・・・ 検証用のデータ<br>
test_survived ・・・ 検証用のデータの答え(Survived)

に分けることができた。

## EDA ~ データからインサイトを得る ~

ここから実際に、分析を行っていく。

まずは、学習用データの中身がどのようなものなのか改めて確認していく。

基本として注目する点は、

1. データ自体のを眺める
2. 欠損値の確認
3. データの特徴を把握する
4. 異常値(外れ値)の有無
5. 数値列間の相関係数
6. その他ドメイン知識や仮説に基づいたデータの探索

ちなみに、データの特徴を掴み、モデル作成のインサイトを得るためにデータをさまざまな角度から見ていくことを、<br>
EDA（Explanatory Data Analysis, 探索データ分析）といいます。

### 1. データ自体を眺める

#### データセットの中身を少し眺める

In [10]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
288,289,1,2,"Hosono, Mr. Masabumi",male,42.0,0,0,237798,13.0000,NaN,S
16,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,NaN,Q
765,766,1,1,"Hogeboom, Mrs. John C (Anna Andrews)",female,51.0,1,0,13502,77.9583,D11,S
297,298,0,1,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.5500,C22 C26,S
99,100,0,2,"Kantor, Mr. Sinai",male,34.0,1,0,244367,26.0000,NaN,S


In [11]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
585,586,1,"Taussig, Miss. Ruth",female,18.0,0,2,110413,79.6500,E68,S
32,33,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.7500,NaN,Q
202,203,3,"Johanson, Mr. Jakob Alfred",male,34.0,0,0,3101264,6.4958,NaN,S
88,89,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0000,C23 C25 C27,S
19,20,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C


ちなみにtrain_data自体のデータ型を調べると、、、

In [15]:
type(train_data)

pandas.core.frame.DataFrame

Pandasというライブラリの`DataFrame`という型となっている。

<公式のドキュメント><br>
https://pandas.pydata.org/pandas-docs/stable/reference/frame.html

#### データセットのサイズを確認する

DataFrame.shape　で　`(行数, 列数)`という出力を得られる。

In [16]:
train_data.shape

(712, 12)

In [17]:
test_data.shape

(179, 11)

### 2. 欠損値の確認

データの中には何らかの理由で、特定の列の値が取得できない場合がある。

その場合、データセットの中に`欠損値`が存在することになる。

例えば、`Age`(年齢)列に欠損値がある場合、そのままでは全体の平均年齢を求めることはできない。

よって、欠損値がデータセットに含まれるのかどうかは大きな問題となる。

In [20]:
# 各列の欠損値の数の合計を求める
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            137
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          544
Embarked         2
dtype: int64

`Age`, `Cabin`, `Embarked`については、欠損値が含まれるので何らかの対処をしなければならないことがわかった。

### 3. データの特徴を把握する

#### 各列のデータ型を確認する

`DataFrame.dtypes`で各列のデータ型を確認できる。

In [23]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

`int64`, `float64`は数値データ
`object`はPythonのobject型。String(文字列)はobjectの一種。

#### 数値データの基本統計量を確認する

`DataFrame.describe()`で各数値列の基本統計量を自動的に計算してくれる。

In [21]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,712.000000,712.000000,712.000000,575.000000,712.000000,712.000000,712.000000
mean,446.654494,0.394663,2.294944,30.017252,0.522472,0.386236,32.659216
std,257.826714,0.489122,0.840318,14.484390,1.081516,0.834206,51.121448
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,224.750000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,446.500000,0.000000,3.000000,29.000000,0.000000,0.000000,14.458300
75%,670.250000,1.000000,3.000000,39.000000,1.000000,0.000000,31.387500
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


数値以外についても`describe`を呼ぶと、最頻値などを確認できる。

In [24]:
train_data.describe(exclude='number')

,Name,Sex,Ticket,Cabin,Embarked
count,712,712,712,168,710
unique,712,2,570,124,3
top,"Hosono, Mr. Masabumi",male,1601,G6,S
freq,1,466,7,4,513


#### カテゴリ値確認する

データの説明などからどれがカテゴリデータかを確認します。

In [25]:
# 全て文字列に変換してから、`describe()`を呼ぶと数値データについてもカテゴリ値かどうか確認できる
train_data.astype('str').describe()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,712,712,712,712,712,712,712,712,712,712,712,712
unique,712,2,3,712,2,85,7,7,570,221,125,4
top,289,0,3,"Hosono, Mr. Masabumi",male,nan,0,0,1601,13.0,nan,S
freq,1,431,387,1,466,137,482,544,7,34,544,513


`unique`行の数値が少ないものはカテゴリ値の場合が多い。

今回は、`Survived`, `Pclass`, `Sex`, `Parch`, `Embarked`は明らかにカテゴリ値。

## 前処理 ~ データクレンジングとフィーチャーエンジニアリング ~

## モデルの作成、学習、推論の実行